In [2]:
import pandas as pd
import os
import numpy as np

In [ ]:
os.listdir(os.curdir)
os.listdir("data")

In [28]:
# buoy_number = 248
buoy_number = 132
SWF = "LANT"
data_fn = 'data/SWF{}_open_waters_buoy{}_sig_wave_height.csv'.format(SWF, buoy_number)
#historic data
# data_fn = 'data/SWF{}_open_waters_buoy{}_sig_wave_height_historic.csv'.format(SWF, buoy_number)
df = pd.read_csv(data_fn)
df.columns = ["date_time", "wave_height_(m)"]
df.tail()

,date_time,wave_height_(m)
2955,1602165600,0.65
2956,1602167400,0.65
2957,1602169200,0.66
2958,1602171000,0.66
2959,1602172800,0.71


In [5]:
def remove_data_NAs(df):
    #this dataset fills missing values with the value 99 (int)
    # not sure what CDIL's N/A value is
    df = df[df['wave_height_(ft)'] != 99]
    return df

def date_time_conversions(df):
    df['date_time'] = pd.to_datetime(df['date_time'], unit='s')
    df = df.set_index("date_time")
    return df

#sea state 1 - 0-.1, mean=0.05
#sea state 2 - 0.1-.5, mean=0.30
#sea state 3 - 0.5 - 1.25, mean = 0.88
#sea state 4 - 1.25 - 2.5, mean = 1.88
#sea state 5 - 2.5 - 4, mean = 3.25
#sea state 6+ - 4<x
def create_sea_state_col(df):
    bins = [0, .1, 0.5, 1.25, 2.5, 4]
    names = [1, 2, 3, 4, 5, 6] #maybe '6+'?
    sea_state_dict = dict(enumerate(names, 1))
    df['sea_state'] = np.vectorize(sea_state_dict.get)(np.digitize(df['wave_height_(m)'], bins))
    return df


In [29]:
#CDIL changes
df = date_time_conversions(df)
sea_state_df = create_sea_state_col(df)
sea_state_df

,wave_height_(m),sea_state
date_time,,
2020-08-07 16:00:00,0.46,2
2020-08-07 16:30:00,0.44,2
2020-08-07 17:00:00,0.44,2
2020-08-07 17:30:00,0.44,2
2020-08-07 18:00:00,0.45,2
2020-08-07 18:30:00,0.44,2
2020-08-07 19:00:00,0.48,2
2020-08-07 19:30:00,0.50,3
2020-08-07 20:00:00,0.51,3


In [30]:
max_df = sea_state_df.groupby(pd.Grouper(freq='D')).max()
max_df.head()

,wave_height_(m),sea_state
date_time,,
2020-08-07,0.62,3
2020-08-08,0.51,3
2020-08-09,0.67,3
2020-08-10,0.85,3
2020-08-11,0.87,3


In [31]:
max_df.tail()

,wave_height_(m),sea_state
date_time,,
2020-10-04,2.11,4
2020-10-05,1.82,4
2020-10-06,1.75,4
2020-10-07,1.13,3
2020-10-08,0.78,3


In [21]:
sea_state_count_df = max_df.groupby(by="sea_state").agg('count')
sea_state_count_df
# pac_df.loc["2020-07-31"]

,wave_height_(m)
sea_state,
2.0,108
3.0,3389
4.0,1530
5.0,118
6.0,9


In [26]:
# max_sea_state_fn = 'data/SWF{}_open_waters_buoy{}_mWH&seastate_2020.csv'.format(SWF, buoy_number)
# sea_state_count_fn = 'data/SWF{}_open_waters_buoy{}_sea_state_counts_2020.csv'.format(SWF, buoy_number)
max_sea_state_fn = 'data/SWF{}_open_waters_buoy{}_mWH&seastate_2020_historic.csv'.format(SWF, buoy_number)
sea_state_count_fn = 'data/SWF{}_open_waters_buoy{}_sea_state_counts_2020_historic.csv'.format(SWF, buoy_number)

In [27]:
max_df.to_csv(max_sea_state_fn)
sea_state_count_df.to_csv(sea_state_count_fn)

In [ ]:
""" doing lant now
-----------------------------------------
"""

In [ ]:
lant_data_fn = 'data/SWFLANT_open_waters_buoy41112_sig_wave_height_2006_2019.csv'
lant_df = pd.read_csv(lant_data_fn)
lant_df.columns = ["date_time", "wave_height_(ft)"]
lant_df.head()

In [ ]:
lant_df = remove_data_NAs(lant_df)
lant_df = col_conversions(lant_df)
lant_sea_state_df = create_sea_state_col(lant_df)
lant_sea_state_df.head()

In [ ]:
lant_max_df = lant_sea_state_df.groupby(pd.Grouper(freq='D')).max()
lant_max_df.head()

In [ ]:
lant_sea_state_count_df = lant_max_df.groupby(by="sea_state").agg('count')
lant_sea_state_count_df
# lant_df.loc["2004-07-31"]

In [ ]:
lant_max_sea_state_fn = 'data/SWFLANT_open_waters_buoy41112_mWH&seastate_2006_2019.csv'
lant_sea_state_count_fn = 'data/SWFLANT_open_waters_buoy41112_sea_state_counts_2006_2019.csv'
lant_max_df.to_csv(lant_max_sea_state_fn)
lant_sea_state_count_df.to_csv(lant_sea_state_count_fn)